# 1. Importa la librería Pyomo 

In [4]:
from pyomo.environ import *

# 2 . Crea un modelo de Pyomo 

In [5]:
# Crear un modelo con Pyomo
model = ConcreteModel()

# 3 . Conjuntos 

In [6]:
model.I = Set()  # Conjunto de productores
model.J = Set()  # Conjunto de centros de acopio
model.K = Set()  # Conjunto de clientes
model.P = Set()  # Conjunto de productos
model.T = Set()  # Conjunto de periodos de tiempo

# 4 .  Parámetros

In [7]:
model.CostoProduccion = Param(model.I, model.P, model.T, within=NonNegativeReals)  # Costos de producción
model.CostoFijoAcopio = Param(model.J, model.T, within=NonNegativeReals)  # Costos fijos de acopio
model.CostoVariableAcopio = Param(model.J, model.P, model.T, within=NonNegativeReals)  # Costos variables de acopio
model.CostoInventario = Param(model.J, model.P, model.T, within=NonNegativeReals)  # Costos de inventario
model.CostoTransporteProducto = Param(model.I, model.J, model.P, model.T, within=NonNegativeReals)  # Costos de transporte de producto
model.CostoTransporteAcopioCliente = Param(model.J, model.K, model.P, model.T, within=NonNegativeReals)  # Costos de transporte de acopio a cliente
model.DemandaCliente = Param(model.P, model.K, model.T, within=NonNegativeReals)  # Demanda de clientes
model.CapacidadProductor = Param(model.I, model.T, within=NonNegativeReals)  # Capacidad de productores
model.CapacidadAlmacenamientoAcopio = Param(model.J, model.T, within=NonNegativeReals)  # Capacidad de almacenamiento de acopio

# 5 . Variables de decisión 

In [8]:
model.CantidadProductoEnviada = Var(model.I, model.J, model.P, model.T, within=NonNegativeReals)  # Cantidad de producto enviada
model.CantidadProductoAlmacenadaAcopio = Var(model.J, model.P, model.T, within=NonNegativeReals)  # Cantidad de producto almacenada en acopio
model.CantidadProductoEnviadaCliente = Var(model.J, model.K, model.P, model.T, within=NonNegativeReals)  # Cantidad de producto enviada al cliente
model.DemandaInsatisfecha = Var(model.P, model.T, within=NonNegativeReals)  # Demanda insatisfecha
model.BinaryOperation = Var(model.J, model.T, within=Binary)  # Variable binaria para activación de centros de acopio

# 6 . Función objetivo 

In [9]:
def objective_rule(model):
    return sum(model.CostoProduccion[i, p, t] * model.CantidadProductoEnviada[i, j, p, t] for i in model.I for j in model.J for p in model.P for t in model.T) + \
           sum(model.CostoFijoAcopio[j, t] * model.BinaryOperation[j, t] for j in model.J for t in model.T) + \
           sum(model.CostoVariableAcopio[j, p, t] * model.CantidadProductoAlmacenadaAcopio[j, p, t] for j in model.J for p in model.P for t in model.T) + \
           sum(model.CostoInventario[j, p, t] * model.CantidadProductoAlmacenadaAcopio[j, p, t] for j in model.J for p in model.P for t in model.T) + \
           sum(model.CostoTransporteProducto[i, j, p, t] * model.CantidadProductoEnviada[i, j, p, t] for i in model.I for j in model.J for p in model.P for t in model.T) + \
           sum(model.CostoTransporteAcopioCliente[j, k, p, t] * model.CantidadProductoEnviadaCliente[j, k, p, t] for j in model.J for k in model.K for p in model.P for t in model.T) + \
           sum(model.DemandaInsatisfecha[p, t] for p in model.P for t in model.T)
model.obj = Objective(rule=objective_rule, sense=minimize)

# 7 . Restricciones de capacidad y demanda

In [10]:
# Restricciones - Capacidad de productores
def capacity_producer_rule(model, i, p, t):
    return model.CantidadProductoEnviada[i, j, p, t] <= model.CapacidadProductor[i, t]
model.capacity_producer = Constraint(model.I, model.P, model.T, rule=capacity_producer_rule)

# Restricciones - Flujo
def flow_rule(model, j, p, t):
    if t == 1:
        return sum(model.CantidadProductoEnviada[i, j, p, t] for i in model.I) == sum(model.CantidadProductoEnviadaCliente[j, k, p, t] for k in model.K) + model.CantidadProductoAlmacenadaAcopio[j, p, t]
    else:
        return sum(model.CantidadProductoEnviada[i, j, p, t] for i in model.I) == sum(model.CantidadProductoEnviadaCliente[j, k, p, t] for k in model.K) + model.CantidadProductoAlmacenadaAcopio[j, p, t] + model.CantidadProductoAlmacenadaAcopio[j, p, t - 1]
model.flow = Constraint(model.J, model.P, model.T, rule=flow_rule)

# Restricciones - Relación de variables binarias y de operación
def binary_operation_rule(model, i, j, p, t):
    return model.CantidadProductoEnviada[i, j, p, t] <= model.CantidadProductoAlmacenadaAcopio[j, p, t] * model.BinaryOperation[j, t]
model.binary_operation_constraint = Constraint(model.I, model.J, model.P, model.T, rule=binary_operation_rule)

# Restricciones - Asignación de envíos desde productores a centros de acopio
def assignment_producer_rule(model, i, p):
    return sum(model.CantidadProductoEnviada[i, j, p, t] for j in model.J for t in model.T) == 1
model.assignment_producer = Constraint(model.I, model.P, rule=assignment_producer_rule)

# Restricciones - Demanda insatisfecha
def demand_unsatisfied_rule(model, p, t):
    return sum(model.CantidadProductoAlmacenadaAcopio[j, p, t] for j in model.J) + model.DemandaInsatisfecha[p, t] == sum(model.DemandaCliente[p, k, t] for k in model.K)
model.demand_unsatisfied_constraint = Constraint(model.P, model.T, rule=demand_unsatisfied_rule)

# Define las capacidades de almacenamiento de acopio
def capacity_acopio_rule(model, j, t):
    return sum(model.CantidadProductoAlmacenadaAcopio[j, p, t] for p in model.P) <= model.CapacidadAlmacenamientoAcopio[j, t]
model.capacity_acopio = Constraint(model.J, model.T, rule=capacity_acopio_rule)

# Define las restricciones de activación de los centros de acopio
def activation_rule(model, j, t):
    return sum(model.CantidadProductoAlmacenadaAcopio[j, p, t] for p in model.P) <= model.CapacidadAlmacenamientoAcopio[j, t] * model.BinaryOperation[j, t]
model.activation = Constraint(model.J, model.T, rule=activation_rule)

# Denife las restricciones - Asignación de envíos desde centros de acopio a clientes
def assignment_acopio_rule(model, j, p, t):
    return sum(model.CantidadProductoEnviadaCliente[j, k, p, t] for k in model.K) == model.DemandaCliente[p, k, t]
model.assignment_acopio = Constraint(model.J, model.P, model.T, rule=assignment_acopio_rule)

# 8 . Resuelve el modelo 

In [11]:
# Resuelve el modelo
solver = SolverFactory('glpk')
results = solver.solve(model)

# 10 . Imprime los resultados 

In [12]:
# Imprime los resultados
if results.solver.termination_condition == TerminationCondition.optimal:
    print("El modelo se resolvió de manera óptima.")
    # Accede a los valores de las variables de decisión si es necesario
    for i in model.I:
        for j in model.J:
            for p in model.P:
                for t in model.T:
                    print(f"CantidadProductoEnviada[{i},{j},{p},{t}] = {model.CantidadProductoEnviada[i,j,p,t].value}")
    # Accede al valor de la función objetivo
    print("Valor de la función objetivo =", model.obj())
else:
    print("El modelo no se resolvió de manera óptima.")

El modelo se resolvió de manera óptima.
Valor de la función objetivo = 0
